In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision.models import densenet121, DenseNet121_Weights

In [ ]:
# Set device to GPU if available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
# Define transforms for data preprocessing
normMean = [0.49139968, 0.48215827, 0.44653124]
normStd = [0.24703233, 0.24348505, 0.26158768]
normTransform = transforms.Normalize(normMean, normStd)

trainTransform = transforms.Compose([
      transforms.RandomCrop(32, padding=4),
      transforms.RandomHorizontalFlip(),
      transforms.ToTensor(),
      normTransform
  ])
testTransform = transforms.Compose([
    transforms.ToTensor(),
    normTransform
])


# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=trainTransform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=testTransform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

# Load pre-trained DenseNet model
model = densenet121(weights=DenseNet121_Weights.DEFAULT)
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-1, momentum=0.9, weight_decay=1e-4)

def adjust_opt(optAlg, optimizer, epoch):
    if optAlg == 'sgd':
        if epoch < 150:
            lr = 1e-1
        elif epoch == 150:
            lr = 1e-2
        elif epoch == 225:
            lr = 1e-3
        else:
            return

        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
    return lr
# Train the model
for epoch in range(300):
    lr = adjust_opt('sgd', optimizer, epoch)  # Adjust learning rate
    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        running_loss += loss.item()
    # print every epoch
    print(f'Epoch: {epoch + 1}, Loss: {running_loss / i:.4f}, Accuracy: {100 * correct / total:.2f}%, lr: {lr:.4f}')
    running_loss = 0.0

print("Training finished!")

# Test the model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')

Files already downloaded and verified
Files already downloaded and verified
Epoch: 1, Loss: 2.9500, Accuracy: 21.31%, lr: 0.1000
Epoch: 2, Loss: 1.7942, Accuracy: 33.26%, lr: 0.1000
Epoch: 3, Loss: 1.6704, Accuracy: 37.59%, lr: 0.1000
Epoch: 4, Loss: 1.5930, Accuracy: 40.65%, lr: 0.1000
Epoch: 5, Loss: 1.5093, Accuracy: 44.18%, lr: 0.1000
Epoch: 6, Loss: 1.4193, Accuracy: 48.53%, lr: 0.1000
Epoch: 7, Loss: 1.3311, Accuracy: 52.10%, lr: 0.1000
Epoch: 8, Loss: 1.2635, Accuracy: 54.99%, lr: 0.1000
Epoch: 9, Loss: 1.2138, Accuracy: 56.60%, lr: 0.1000
Epoch: 10, Loss: 1.2537, Accuracy: 55.10%, lr: 0.1000
Epoch: 11, Loss: 1.1398, Accuracy: 59.15%, lr: 0.1000
Epoch: 12, Loss: 1.0804, Accuracy: 61.35%, lr: 0.1000
Epoch: 13, Loss: 1.0451, Accuracy: 62.67%, lr: 0.1000
Epoch: 14, Loss: 0.9934, Accuracy: 64.78%, lr: 0.1000
Epoch: 15, Loss: 0.9582, Accuracy: 66.34%, lr: 0.1000
Epoch: 16, Loss: 0.9292, Accuracy: 67.62%, lr: 0.1000
Epoch: 17, Loss: 0.9053, Accuracy: 68.38%, lr: 0.1000
Epoch: 18, Loss